In [37]:
from __future__ import division, print_function, absolute_import

In [39]:
import numpy as np
from PIL import Image
from PIL import ImageFile

Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True

import cv2
import glob
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pandas as pd

def ListImages():    
    # imagelist :: list of full paths of images
    # imagelist_wo :: list of image filenames
    
    imagelist = []
    imagelist_wo = []
    folder = 'train_0/'
    # NOTE!!!!!!
    # ADJUST FILEPATH TO SUIT YOUR PURPOSES
    for filepath in glob.iglob(folder+'*.jpg'):
        imagelist.append(filepath)        
        imagelist_wo.append(filepath[len(folder):]) #THIS WILL ALSO CHANGE!
    return imagelist, imagelist_wo


def CreateDesignVector(imagelist,X=[],n=256):   
    # n :: number of pixels per side
    for filepath in imagelist:
        img = Image.open(filepath).convert('RGB')
        #height, width, channels = img.shape
        #if (height*width<85000):
        arr = np.array(img)
        arr = cv2.resize(arr, (n, n))
        arr = arr.ravel()
        X.append(arr)
    return X

In [41]:
# Read CSV file with filenames, genres, etc.
info = pd.read_csv("train_info.csv")
imagelist, imagelist_wo = ListImages()
train_info = info.loc[info['filename'].isin(imagelist_wo)]

In [42]:
#save X without augmentation 
X = CreateDesignVector(imagelist)
#np.save('X_256.dat', X)

KeyboardInterrupt: 

In [ ]:
#save Y without augmentation 
sorterIndex = dict(zip(imagelist_wo,range(len(imagelist_wo))))
train_info['filename_rank'] = train_info['filename'].map(sorterIndex)
train_info = train_info.sort_values(['filename_rank'], ascending = [True])
Y = train_info['genre'].values
#np.save('Y_genre.dat', X)

In [46]:
X = np.load('X_256.dat.npy')


In [44]:
Y = np.load('Y_genre.dat.npy')

In [47]:
X.shape, Y.shape

((79433, 196608), (79433,))

In [50]:
def label_class_to_Integer(Y):
    #dictY = dict(zip(np.unique(Y),range(len(np.unique(Y)))))
    #dictY = dict(zip(range(len(np.unique(Y.astype(str)))),np.unique(Y.astype(str))))   
    dictY = dict(zip(np.unique(Y.astype(str)),range(len(np.unique(Y.astype(str))))))   

    num_classes = len(dictY)
    for i in range(len(Y)):
        Y[i] = dictY[Y[i]]
        
    return Y.astype(int), num_classes, dictY

In [51]:
Y = Y.astype(str)
Y, num_classes, dictY = label_class_to_Integer(Y)

In [52]:
#split data 
percentage = 0.8
ntrain = len(X)
X_train = X[:ntrain]
Y_train = Y[:ntrain]
X_valid = X[ntrain:]
Y_valid = Y[ntrain:]

In [53]:
Y

array([28, 31, 28, ...,  0, 15, 19])

In [56]:
def getFrequency_and_augment(X, Y, num_classes, dsrd_count):

    listCount = [0]*num_classes
    
    #first combine the matrix X and Y 
    Y = Y.reshape((len(Y), 1))
    XX = np.append(X, Y,1)

    
    for i in range(len(XX)):
        listCount[XX[i, -1]] +=1
        
    
    newSample =[]
    for i in range (len(XX)):
        
        freq = listCount[XX[i, -1]]

        nn = dsrd_count//freq
        
        if nn>0:
            for j in range(nn):
                newSample.append(XX[i, :])
        else:
            newSample.append(XX[i, :])
           
    print (len(newSample))
    return np.array(newSample)

In [57]:
#We have X and Y, now we do augmentation because the dataset is imbalanced
#For now we simply copy the data without any transformation 
#lets try 1000 first 
#ONLY Augment training data not the validation/test! 
XX = getFrequency_and_augment(X_train, Y_train, num_classes, 1000)
#now need to shuffle the data 
np.random.shuffle(XX)


#save training data with augmentation
np.save('X_256_augment_train', XX[:, :-1])
np.save('Y_genre_augment_train', XX[:, -1])

np.save('X_256_validation', X_valid)
np.save('Y_genre_validation', Y_valid)

MemoryError: 